In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef('Series')]
[<class 'str'>, <class 'float'>, <class 'bool'>]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta')]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval')]
[<class 'str'>, <class 'float'>, <class 'bool'>, ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval'), <class 'numpy.datetime64'>, <class 'numpy.timedelta64'>, <class 'datetime.datetime'>]
[ForwardRef('Timestamp'), <class 'datetime.datetime'>, <class 'numpy.datetime64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[ForwardRef('Timedelta'), <class 'datetime.timedelta'>, <class 'numpy.timedelta64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[<class 'str'>, <class 'datetime.tzinfo'>]
[<class 'str'>, <class 'int'>]
[typing.Hashable, typing.Sequence[typing.Ha

[19:42:04] Initializing Normalizer


In [3]:
#Read the influx data (S: substrate, NS:non-substrate)
influx_chembl =  pd.read_csv('raw_data/influx/influx_ChEMBL_query.csv', delimiter = ',')
LAT1_S = Chem.SDMolSupplier('raw_data/influx/LAT1_influx_metrabase_S.sdf')
#No LAT1_NS because there was no result
OATP1A2_S = Chem.SDMolSupplier('raw_data/influx/OATP1A2_influx_metrabase_S.sdf')
OATP1A2_NS = Chem.SDMolSupplier('raw_data/influx//OATP1A2_influx_metrabase_NS.sdf')

In [4]:
#Create target lists for data and UniProt Accession
influx_targets_S_data = [LAT1_S,OATP1A2_S]
influx_targets_S_UniProt= ['Q01650','P46721']

influx_targets_NS_data = [OATP1A2_NS]
influx_targets_NS_UniProt = ['P46721']

In [4]:
#Assign classes: influx_chembl['status_influx'] = influx_chembl['pchembl_value'].apply(lambda pchembl: 'Substrate' if pchembl > 5 else 'Non-substrate')

def compare_values(row):
    if row['pchembl_value'] > 5:
        return 'Substrate'
    elif row['pchembl_value'] <= 5:
        return 'Non-substrate'


influx_chembl['status_influx'] = influx_chembl.apply(compare_values, axis=1)


influx_chembl.head(10)

,molregno,pchembl_value,canonical_smiles,standard_inchi_key,accession,status_influx
0,398458,4.96,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(cc(O...,HJYYPODYNSCCOU-ODRIEIDWSA-N,P46721,Non-substrate
1,365189,4.29,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...,JQXXHWHPUNPDRT-WLSIYKJHSA-N,P46721,Non-substrate
2,11143,4.18,CC(C)N(CCC(C(N)=O)(c1ccccc1)c1ccccn1)C(C)C,UVTNFZQICZKOEM-UHFFFAOYSA-N,Q96FL8,Non-substrate
3,86042,4.58,OCCN(CCO)c1nc(N2CCCCC2)c2nc(N(CCO)CCO)nc(N3CCC...,IZEKFCXSFNUWAM-UHFFFAOYSA-N,Q96FL8,Non-substrate
4,605,5.00,CN(C)CCCN1c2ccccc2CCc2ccccc21,BCGWQEUPMDMJNV-UHFFFAOYSA-N,Q96FL8,Non-substrate
5,9333,5.96,Nc1c2c(nc3ccccc13)CCCC2,YLJREFDVOIBQDA-UHFFFAOYSA-N,Q96FL8,Substrate
6,77779,4.19,Cc1ccccc1C(OCCN(C)C)c1ccccc1,QVYRGXJJSLMXQH-UHFFFAOYSA-N,Q96FL8,Non-substrate
7,3183,6.82,Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,FELGMEQIXOGIFQ-UHFFFAOYSA-N,Q96FL8,Substrate
8,1620,5.24,CN/C(=N\CCSCc1nc[nH]c1C)NC#N,AQIXAKUUQRKLND-UHFFFAOYSA-N,Q96FL8,Substrate
9,3965,6.39,N=C(N)c1ccc(OCCCCCOc2ccc(C(=N)N)cc2)cc1,XDRYMKDFEDOLFX-UHFFFAOYSA-N,Q96FL8,Substrate


In [6]:
influx_df = influx_chembl

#Add substrates from Metrabase data
for i in range(len(influx_targets_S_data)): #Iterate through the list of SD files

    for mol in influx_targets_S_data[i]: #Read molecules out of SD file

        smiles = Chem.MolToSmiles(mol)
        inchi_str = inchi.MolToInchi(mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)

        new_row = {'Molregno': 'Unknown', 'pchembl_value': 'Unknown', 
                   'canonical_smiles': smiles, 'standard_inchi_key': inchi_key, 
                   'accession': influx_targets_S_UniProt[i],
                   'status_influx': 'Substrate'}
        influx_df.loc[len(influx_df)] = new_row

#Add non-substrates from Metrabase data
for i in range(len(influx_targets_NS_data)):

    for mol in influx_targets_NS_data[i]:

        smiles = Chem.MolToSmiles(mol)
        inchi_str = inchi.MolToInchi(mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)

        new_row = {'Molregno': 'Unknown', 'pchembl_value': 'Unknown', 
                   'canonical_smiles': smiles, 'standard_inchi_key': inchi_key, 
                   'accession': influx_targets_NS_UniProt[i],
                   'status_influx': 'Non-substrate'}
        
        influx_df.loc[len(influx_df)] = new_row

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Proton(s) added/removed

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Proton(s) added/removed

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Omitted undefined stereo

[19:43:06] WARNING: Charges were rearranged



In [8]:
#Keep the relevant columns

df=influx_df.rename(columns={'canonical_smiles':'SMILES_raw'})

df = df[['SMILES_raw','status_influx']]


206


,SMILES_raw,status_influx
0,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(cc(O...,Non-substrate
1,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(c(/C...,Non-substrate
2,CC(C)N(CCC(C(N)=O)(c1ccccc1)c1ccccn1)C(C)C,Non-substrate
3,OCCN(CCO)c1nc(N2CCCCC2)c2nc(N(CCO)CCO)nc(N3CCC...,Non-substrate
4,CN(C)CCCN1c2ccccc2CCc2ccccc21,Non-substrate


In [10]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [39]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [40]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [41]:
def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [42]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_influx'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [ ]:
def class_code(df):
    df['status_influx'] = df['status_influx'].replace({'Substrate': 1, 'Non-substrate': 0})

    return df

In [43]:
df = remove_nan_smiles(df)

df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_influx_train.csv')


An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[21:02:43] WARNING: Omitted undefined stereo

[21:02:43] WARNING: Omitted undefined stereo

[21:02:43] WARNING: Omitted undefined stereo

[21:02:43] WARNING: Omitted undefined stereo

[21:02:43] WARNING: Charges were rearranged

[21:02:44] WARNING: Omitted undefined stereo

[21:02:44] WARNING: Charges were rearranged

[21:02:44] WARNING: Omitted undefined stereo

[21:02:44] WARNING: Omitted undefined stereo

[21:02:44] WARNING: Charges were rearranged

[21:02:44] WARNING: Omitted undefined stereo

[21:02:45] WARNING: Charges were rearranged

[21:02:45] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[21:02:45] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


...

[21:02:58] WARNING: Omitted undefined stereo

[21:02:58] WARNING: Omitted undefined stereo

[21:02:58] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[21:02:58] WARNING: Charges were rearranged; Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[21:02:59] WARNING: Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_18124\2591752290.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
